In [7]:
import pdfplumber
import os

In [8]:
file_path = 'book.pdf'
pdf = pdfplumber.open(file_path) 

In [9]:
i = 7
chars = pdf.pages[i].chars
x1 = max([char['x1'] for char in chars])
print('The max x1: '+str(x1))

The max x1: 367.604


In [10]:
i = 8
chars = pdf.pages[i].chars
x1 = max([char['x1'] for char in chars])
print('The max x1: '+str(x1))

The max x1: 364.725


In [11]:
i = 28
chars = pdf.pages[i].chars
print(chars[0]['text'])
space = [chars[i-1]['y0']-chars[i]['y1'] for i in range(1,len(chars)) if chars[i-1]['y0']-chars[i]['y1']>4.5]
space_list = [chars[i]['text'] for i in range(1,len(chars)) if chars[i-1]['y0']-chars[i]['y1']>4.5]
''.join(space_list)

2


'cACPCCcA'

In [12]:
chars = []
for page in pdf.pages[7:166]:
    for char in page.chars:
        chars.append(char)
width_unique = set([char['height'] for char in chars])
print('The unique heights: '+str(width_unique))

The unique heights: {Decimal('9.000'), Decimal('10.000'), Decimal('6.300'), Decimal('14.000')}


In [13]:
headers = [i for i in range(len(chars)) if chars[i]['height'] == 10.000]
separate_header = [(0, headers[0])]
for i in range(len(headers)-1):
    if headers[i]+1 != headers[i+1]:
        separate_header.append((headers[i],headers[i+1]))
separate_header.append((headers[-1], len(chars)))

In [14]:
header_list = []
body_list = []
for i in range(1, len(separate_header)):
    min_header = separate_header[i-1][1]
    max_header = separate_header[i][0]+1 
    min_body = separate_header[i][0]+1
    max_body = separate_header[i][1] 
    header_list.append(''.join([chars[i]['text'] for i in range(min_header, max_header)]))
    body_list.append(''.join([('\n'+chars[i]['text']) if chars[i-1]['y0']-chars[i]['y1']>4.5
                              else (''+chars[i]['text'])
                              for i in range(min_body, max_body)]))

In [15]:
import pandas as pd
d = {'header':header_list,'body':body_list}
df = pd.DataFrame(d)

In [16]:
import docx2txt
# read in word file
result = docx2txt.process("terms.docx")
result = '\n'.join([line for line in result.split('\n') if line])
import re
result = re.sub(r'( \(.*\))', '', result)
result = re.sub(r'(/| and )', '\n', result)
result = re.sub(r'(process)\n', 'processes\n', result)
result = re.sub(r'(,.*)\n', '\n', result).lower()

In [17]:
df['header'] = df['header'].str.replace(r'( \(.*\))', '')

In [18]:
df['body'] = df['body'].str.replace(r'(\W)([a-z]+ \d{2,3})(\n|$)', r'\1 \3',regex=True)
df['body'] = df['body'].str.replace(r'(\W)(\d{2,3} [a-z]+)(\n|$)', r'\1 \3',regex=True)

In [19]:
result_df = df[df['header'].isin(result.split('\n'))]
list_headers = result_df['header'].to_list()

In [21]:
result_df.to_csv('result.csv')

In [22]:
[header for header in result.split('\n')if header not in list_headers]

['discursive',
 'ideational functional',
 'interpersonal functional',
 'textual function',
 'appraisal system']

In [23]:
import pandas as pd
df = pd.read_csv('result.csv')

In [24]:
from IPython.display import HTML
s  = '<script type="text/Javascript">'
s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
s += 'win.document.body.innerHTML = \'' + result_df.to_html().replace("\n",'\\') + '\';'
s += '</script>'

# Show in new Window
HTML(s)

In [31]:
with open('result.txt', 'w', encoding = 'utf-8') as f:
    for i in range(df.shape[0]):
        f.write(df.header[i])
        f.write(df.body[i])
        f.write('\n')

In [32]:
from docx import Document
from docx.shared import Inches

document = Document()

for i in range(df.shape[0]):
    document.add_heading(df.header[i], 0)
    document.add_paragraph(df.body[i])
document.save('result.docx')